In [1]:
%pip install -q bitsandbytes datasets accelerate loralib
%pip install -q git+https://github.com/huggingface/transformers.git@main 
%pip install -q git+https://github.com/huggingface/peft.git

^C
ERROR: Operation cancelled by user


## base model & lora adapters

In [4]:
import torch
import torch.nn as nn
import bitsandbytes as bnb 
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such file), '/Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.

/Users/drayl/miniconda3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:

model_path = '/Users/drayl/Downloads/mlc-llm/dist/models/RedPajama-INCITE-Chat-3B-v1'
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    load_in_8bit=False, 
    device_map={"": 'cpu'}
)
model = model.float()

tokenizer = AutoTokenizer.from_pretrained(model_path)
# 这里是新增加的
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [6]:
# model.config
AutoConfig.from_pretrained(model_path)

GPTNeoXConfig {
  "_name_or_path": "/Users/drayl/Downloads/mlc-llm/dist/models/RedPajama-INCITE-Chat-3B-v1",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "rotary_emb_base": 10000,
  "rotary_pct": 1.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.0.dev0",
  "use_cache": true,
  "use_parallel_residual": false,
  "vocab_size": 50432
}

In [7]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 2560)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=2560, out_features=50432, bias=False)
)

In [8]:
# model.transformer.word_embeddings
model.get_input_embeddings()

Embedding(50432, 2560)

In [9]:
tokenizer

GPTNeoXTokenizerFast(name_or_path='/Users/drayl/Downloads/mlc-llm/dist/models/RedPajama-INCITE-Chat-3B-v1', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True)

### freeze original weights

In [11]:
list(model.parameters())[0].dtype

torch.float32

In [12]:
for i, param in enumerate(model.parameters()):
    param.requires_grad = False  # freeze the model - train adapters later
#     print(i, 'param.requires_grad = False')
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)
#         print(i, 'ndim == 1, torch.float16 to torch.float32')

In [13]:
# reduce number of stored activations
model.gradient_checkpointing_enable()  
model.enable_input_require_grads()

In [14]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): 
        return super().forward(x).to(torch.float32)
# model.lm_head = CastOutputToFloat(model.lm_head)
model.embed_out = CastOutputToFloat(model.embed_out)


### LoRa Adapters

In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
from peft import LoraConfig, get_peft_model 
config = LoraConfig(
    r=16, #low rank
    lora_alpha=32, #alpha scaling， scale lora weights/outputs
    # target_modules=["q_proj", "v_proj"], #if you know the 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

In [17]:
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 5242880 || all params: 2781107200 || trainable%: 0.18851772416395887


In [18]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50432, 2560)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=2560, out_features=7680, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=7680, bias=False)
                )
  

## pipeline

### data

In [19]:
import transformers
from datasets import load_dataset
dataset = load_dataset("Abirate/english_quotes")

Found cached dataset json (/Users/drayl/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 97.32it/s]


In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [21]:
dataset['train']

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})

In [22]:
dataset['train'].to_pandas()

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[be-yourself, gilbert-perreira, honesty, inspi..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[best, life, love, mistakes, out-of-control, t..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[human-nature, humor, infinity, philosophy, sc..."
3,"“So many books, so little time.”",Frank Zappa,"[books, humor]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[books, simile, soul]"
...,...,...,...
2503,“Morality is simply the attitude we adopt towa...,"Oscar Wilde,","[morality, philosophy]"
2504,“Don't aim at success. The more you aim at it ...,"Viktor E. Frankl,","[happiness, success]"
2505,"“In life, finding a voice is speaking and livi...",John Grisham,[inspirational-life]
2506,"“Winter is the time for comfort, for good food...",Edith Sitwell,"[comfort, home, winter]"


In [23]:
dataset['train']['quote'][:4]

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
 '“So many books, so little time.”']

In [24]:
dataset['train']['author'][:4]

['Oscar Wilde', 'Marilyn Monroe', 'Albert Einstein', 'Frank Zappa']

In [25]:
dataset['train'][:4]

{'quote': ['“Be yourself; everyone else is already taken.”',
  "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
  "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
  '“So many books, so little time.”'],
 'author': ['Oscar Wilde', 'Marilyn Monroe', 'Albert Einstein', 'Frank Zappa'],
 'tags': [['be-yourself',
   'gilbert-perreira',
   'honesty',
   'inspirational',
   'misattributed-oscar-wilde',
   'quote-investigator'],
  ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst'],
  ['human-nature',
   'humor',
   'infinity',
   'philosophy',
   'science',
   'stupidity',
   'universe'],
  ['books', 'humor']]}

In [26]:
str(dataset['train']['tags'][0])

"['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"

In [27]:
def merge(row):
    row['prediction'] = row['quote'] + ' ->: ' + str(row['tags'])
    return row
dataset['train'] = dataset['train'].map(merge)

Loading cached processed dataset at /Users/drayl/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-46754fe75a1dd0c9.arrow


In [28]:
dataset['train']['prediction'][:5]

["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [29]:
dataset['train'][4]

{'quote': '“A room without books is like a body without a soul.”',
 'author': 'Marcus Tullius Cicero',
 'tags': ['books', 'simile', 'soul'],
 'prediction': "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"}

In [30]:
tokenizer(dataset['train']['prediction'][:4])

{'input_ids': [[1628, 4678, 4834, 28, 4130, 2010, 310, 2168, 2668, 1425, 5204, 27, 14412, 1257, 14, 12550, 1286, 1383, 686, 44626, 6291, 14, 468, 42800, 1383, 686, 25224, 14197, 1383, 686, 968, 5378, 1050, 1383, 686, 24418, 1595, 3567, 14, 375, 5546, 14, 88, 6227, 1383, 686, 21049, 14, 24889, 22853, 6038], [1628, 42, 1353, 27915, 13, 34456, 285, 247, 1652, 46519, 15, 309, 1056, 16503, 13, 309, 717, 562, 273, 1453, 285, 387, 2069, 1892, 281, 6016, 15, 1292, 604, 368, 476, 626, 6016, 479, 387, 619, 9065, 13, 840, 368, 2119, 347, 7085, 1053, 626, 17337, 479, 387, 619, 1682, 1425, 5204, 27, 14412, 14461, 1383, 686, 8206, 1383, 686, 26617, 1383, 686, 33542, 1582, 1383, 686, 483, 14, 1171, 14, 8519, 1383, 686, 33024, 1383, 686, 42108, 296, 6038], [1628, 7910, 1841, 403, 11968, 27, 253, 10325, 285, 1966, 11339, 414, 28, 285, 309, 1353, 417, 2119, 670, 253, 10325, 1425, 5204, 27, 14412, 13961, 14, 36791, 1383, 686, 73, 15502, 1383, 686, 43723, 1383, 686, 545, 5463, 7868, 1383, 686, 21559, 1383

### tokenize

In [31]:
dataset = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [32]:
# 'input_ids', 'attention_mask'
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

### training

In [33]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [34]:
trainer = Trainer(
    model=model, 
    train_dataset=dataset['train'],
    args=TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=200, 
        learning_rate=2e-4, 
        fp16=False, #必须移除，只支持CUDA
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  
trainer.train()

/Users/drayl/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/200 [01:36<5:20:55, 96.76s/it]

{'loss': 2.438, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  1%|          | 2/200 [02:43<4:21:52, 79.36s/it]

{'loss': 2.9013, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}


  2%|▏         | 3/200 [04:53<5:35:42, 102.25s/it]

...

100%|█████████▉| 199/200 [37:32:37<41:13, 2473.23s/it]

{'loss': 1.784, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.27}


100%|██████████| 200/200 [37:50:52<00:00, 681.26s/it] 

{'loss': 1.4555, 'learning_rate': 0.0, 'epoch': 1.28}
{'train_runtime': 136252.6176, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.001, 'train_loss': 1.8822336348891258, 'epoch': 1.28}


TrainOutput(global_step=200, training_loss=1.8822336348891258, metrics={'train_runtime': 136252.6176, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.001, 'train_loss': 1.8822336348891258, 'epoch': 1.28})

### inference

In [35]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/Users/drayl/miniconda3/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/drayl/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/Users/drayl/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")




 “Training models with PEFT and LoRa is cool” ->: ขอบคุณครับ ฉันเป็นชาวประชาชนชาวประชาชน�


In [36]:
 
batch = tokenizer("“An important paradigm of natural language processing consists of large-scale pre-training on general domain data and adaptation to particular tasks or domains.” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 “An important paradigm of natural language processing consists of large-scale pre-training on general domain data and adaptation to particular tasks or domains.” ->:  ['adaptation', 'domain-specific', 'general', 'language-processing', 'pre-training', 'task-specific'], ['language-processing'], ['natural-language-processing'], ['pre-


In [36]:
trainer.data_collator

DataCollatorForLanguageModeling(tokenizer=GPTNeoXTokenizerFast(name_or_path='/Users/drayl/Downloads/mlc-llm/dist/models/RedPajama-INCITE-Chat-3B-v1', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')